# Computation of the CTA sensitivity

## Introduction

This notebook explains how to derive the CTA sensitivity for a point-like IRF at a fixed zenith angle and fixed offset. The significativity is computed for the 1D analysis (On-OFF regions) and the LiMa formula.

We will be using the following Gammapy classes:

* gammapy.irf.CTAIrf
* [gammapy.spectrum.SensitivityEstimator](https://docs.gammapy.org/dev/api/gammapy.spectrum.SensitivityEstimator.html)

## Setup
As usual, we'll start with some setup ...

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from astropy.coordinates import Angle
from gammapy.spectrum import SensitivityEstimator
from gammapy.irf import load_cta_irfs

ImportError: cannot import name 'CTAPerf'

## Load IRFs

First load the CTA IRFs.

In [3]:
filename = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"
irfs = load_cta_irfs(filename)

NameError: name 'CTAPerf' is not defined

## Extract reduced IRFs

We extract from the full 3D IRFs the relevant effective area and energy dispersion at the specific offset we are interested in.

In [ ]:
offset = Angle("0.7 deg")
arf = irfs['aeff'].to_effective_area_table(offset)
rmf = irfs['edisp'].to_energy_dispersion(offset)

## Compute sensitivity

Choose a few parameters, then run the sentitivity computation.

In [4]:
sensitivity_estimator = SensitivityEstimator(irf=irf, livetime="5h")
sensitivity_estimator.run()

NameError: name 'SensitivityEstimator' is not defined

## Results

The results are given as an Astropy table.

In [5]:
# Show the results table
sensitivity_estimator.results_table

NameError: name 'sensitivity_estimator' is not defined

In [6]:
# Save it to file (could use e.g. format of CSV or ECSV or FITS)
# sensitivity_estimator.results_table.write('sensitivity.ecsv', format='ascii.ecsv')

In [7]:
# Plot the sensitivity curve
t = sensitivity_estimator.results_table

is_s = t["criterion"] == "significance"
plt.plot(
    t["energy"][is_s],
    t["e2dnde"][is_s],
    "s-",
    color="red",
    label="significance",
)

is_g = t["criterion"] == "gamma"
plt.plot(
    t["energy"][is_g], t["e2dnde"][is_g], "*-", color="blue", label="gamma"
)

plt.loglog()
plt.xlabel("Energy ({})".format(t["energy"].unit))
plt.ylabel("Sensitivity ({})".format(t["e2dnde"].unit))
plt.legend();

NameError: name 'sensitivity_estimator' is not defined

## Exercises

* Also compute the sensitivity for a 20 hour observation
* Compare how the sensitivity differs between 5 and 20 hours by plotting the ratio as a function of energy.